## Regresyon

Tikhonov, Polinom, Basis Function
Validasyon Görselleştirmesi
Gradient Tree Boosting, MLP

In [ ]:
#Ortamı hazırlayalım
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Doğrusal regresyon

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

x = np.linspace(-1,1,50)
X = x[:,np.newaxis]
y = 2*x+1 + np.random.normal(0,0.25,len(x))
lr = LinearRegression(fit_intercept = True)
lr.fit(X,y)
ylr = lr.predict(X)
plt.scatter(X,y)
plt.plot(X,ylr,'r')
plt.show()

print('Ortalama Karesel Hata: ', mean_squared_error(y,ylr))
print('R2 puanı: ', lr.score(X,y))


Scikit-learn kestirim için tasarlanmış. O yüzden istatistiksel hesaplamalara girmiyor. Ancak regresyon parametrelerinin güven aralıkları öznitelik seçimi için kullanılabilir. Bu özelliğe ihtiyaç varsa bir yol gerekli kodu yazmak. Ancak **statsmodel** kütüphanesinde bu özellik mevcut.

### Polinom Regresyonu

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

#Pipeline adımlarını hazırlayalım. 5. dereceden seçelim
# include_bias=True yazıp, fit_intercept=False da olabilirdi
stepsPoly = [('poly', PolynomialFeatures(5, include_bias=False)), 
             ('lr', LinearRegression(fit_intercept=True)) ]

#make_pipeline ile de yapabilirdik
pipePoly = Pipeline(stepsPoly)
xP = np.linspace(0.1,7,100)    
#Normal gürültü yerine tekdüze (uniform) gürültü yaratıp biraz daha zorlayalım
yP = np.log(xP) + np.sin(xP) + np.random.uniform(-0.5,0.5,len(xP))#np.random.normal(0,0.25,len(xP))
             
XP = xP[:,np.newaxis]
pipePoly.fit(XP,yP)
print('R2: ', pipePoly.score(XP,yP))
yPredP = pipePoly.predict(XP)

plt.scatter(xP,yP)
plt.plot(xP,yPredP,'r')
plt.show()

Veri polinom olmasa da öğrenebiliyor. Ancak veri görmediğimiz yerden gelseydi bu iyi çalışmaz, o yüzden dikkatli olmak gerekli

In [ ]:
xP2 = np.linspace(6,10,100)
yP2 = np.log(xP2) + np.sin(xP2)
XP2 = xP2[:,np.newaxis]
yPredP2 = pipePoly.predict(XP2)
plt.plot(xP2,yP2)
plt.plot(xP2,yPredP2,'r')
plt.show()


### Başka Fonksiyonlarla Regresyon

Polinomlar dışında da fonksiyon kullanmak mümkün. Gaussian RBF özniteliklerine bakalım. 

In [ ]:
class GaussianRbfFeatures():
    def __init__(self,merkezSayisi=10,genlikKatsayisi=1.0):
        #Merkezleri ve genlikleri veriye göre otomatik ayarlıyoruz
        self.k = merkezSayisi
        self.h = genlikKatsayisi
    
    @staticmethod
    def _rbf(x,c,h):
        return np.exp(-np.sum(((x-c)/h)**2, axis=1))
    
    def fit(self,X,y=None):
        self.merkezler_ = np.linspace(X.min(), X.max(), self.k)
        self.genlikler_ = self.h*(self.merkezler_[1]-self.merkezler_[0])
        return self
    
    def transform(self,X):
        return self._rbf(X[:, :, np.newaxis], self.merkezler_, self.genlikler_)
    
    def fit_transform(self, X, y=None):
        self.fit(X,y)
        return self.transform(X)
        
stepsRBF = [('rbf', GaussianRbfFeatures(20)), 
            ('lr', LinearRegression(fit_intercept=True)) ]

#make_pipeline ile de yapabilirdik
pipeRBF = Pipeline(stepsRBF)
             
pipeRBF.fit(XP,yP)
print('R2: ', pipeRBF.score(XP,yP))
yPredRBF = pipeRBF.predict(XP)

plt.scatter(xP,yP)
plt.plot(xP,yPredRBF,'r')
plt.show()
            

Kendi tanımladığımız öznitelikler üzerinden de hiperparametre arayabiliriz. Ancak bunun öznitelik çıkartan transformerımıza bir takım değişikler yapmamız gerekli

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class GaussianRbfFeaturesPipeline(BaseEstimator,TransformerMixin):
    def __init__(self,merkezSayisi=10,genlikKatsayisi=1.0):
        #Merkezleri ve genlikleri veriye göre otomatik ayarlıyoruz
        self.k = merkezSayisi
        self.h = genlikKatsayisi
    
    @staticmethod
    def _rbf(x,c,h):
        return np.exp(-np.sum(((x-c)/h)**2, axis=1))
    
    def fit(self,X,y=None):
        self.merkezler_ = np.linspace(X.min(), X.max(), self.k)
        self.genlikler_ = self.h*(self.merkezler_[1]-self.merkezler_[0])
        return self
    
    def transform(self,X):
        return self._rbf(X[:, :, np.newaxis], self.merkezler_, self.genlikler_)
    
    def fit_transform(self, X, y=None):
        self.fit(X,y)
        return self.transform(X)
    
    def get_params(self, deep=True):
        return {'merkezSayisi':self.k,'genlikKatsayisi':self.h}
    
    def set_params(self, **params):
        self.k = params['merkezSayisi']
        self.h = params['genlikKatsayisi']

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.pipeline import make_pipeline

cv = KFold(n_splits=5, shuffle=True)
paramsRBF = {'rbf__merkezSayisi': np.arange(5,20), 
             'rbf__genlikKatsayisi': np.linspace(0.25,5.0,100)}
stepsRBF2 = [('rbf', GaussianRbfFeaturesPipeline()), 
             ('lr', LinearRegression(fit_intercept=True)) ]

pipeRBF2 = Pipeline(stepsRBF2)

rgbCV = RandomizedSearchCV(pipeRBF2, paramsRBF, cv=cv, n_iter=50)
rgbCV.fit(XP,yP)
print('R2: ', rgbCV.score(XP,yP))
yPredRBF2 = rgbCV.predict(XP)
print(rgbCV.best_params_ )
plt.scatter(xP,yP)
plt.plot(xP,yPredRBF2,'r')
plt.show()

Öznitelikler ile doğrusalsızlık ekleyerek daha iyi sonuçlar alabiliyoruz. Ancak burada overfitting durumuna dikkat etmemiz gerekiyor. Bunu hiperparametre taraması yaparak azalttık. Ancak bu tarama süreci vakit alabilir. Öznitelik tanımlamadan bile overfitting mümkün. 

Overfittingi özniteliklerden bağımsız bir şekilde regülarizasyon ile azaltabiliriz. Bunun bir yolu *Ridge Regression*. Eğitim performansını düşürse de overfittinge karşı daha başarılı.

In [ ]:
from sklearn.linear_model import Ridge

pipeRBF3 = make_pipeline(GaussianRbfFeatures(50,2.), 
                         LinearRegression(fit_intercept=True))
pipeRBF3.fit(XP, yP)
print('R2: ', pipeRBF3.score(XP,yP))
yPredRBF3 =  pipeRBF3.predict(XP)
plt.scatter(xP,yP)
plt.plot(xP,yPredRBF3,'r')
plt.show()

print(pipeRBF3.named_steps['linearregression'].coef_, pipeRBF3.named_steps['linearregression'].intercept_ )

pipeRbfRidge = make_pipeline(GaussianRbfFeatures(50,2.), 
                             Ridge(alpha=2.0,fit_intercept=True))
pipeRbfRidge.fit(XP, yP)
print('R2: ', pipeRbfRidge.score(XP,yP))
yPredRbfRidge =  pipeRbfRidge.predict(XP)
plt.scatter(xP,yP)
plt.plot(xP,yPredRbfRidge,'r')
plt.show()

print(pipeRbfRidge.named_steps['ridge'].coef_, pipeRbfRidge.named_steps['ridge'].intercept_ )

Bir de Lasso regresyonu var. Bu yöntem, bazı öznitelikleri 0 olmaya zorluyor.

In [ ]:
from sklearn.linear_model import Lasso

pipeRbfLasso = make_pipeline(GaussianRbfFeatures(50,2.), 
                             Lasso(alpha=0.001, fit_intercept=True))
pipeRbfLasso.fit(XP, yP)
print('R2: ', pipeRbfLasso.score(XP,yP))
yPredRbfLasso =  pipeRbfLasso.predict(XP)
plt.scatter(xP,yP)
plt.plot(xP,yPredRbfLasso,'r')
plt.show()

print(pipeRbfLasso.named_steps['lasso'].coef_, pipeRbfLasso.named_steps['lasso'].intercept_ )

Scikit-Learn içinde başka regresyon algoritmaları da mevcut. Bunlardan bir kaçını scikit-learn içindeki boston veri seti ile deneyelim

In [ ]:
from sklearn import datasets

from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import time
seed = int(time.time())

def quickTest(X, y, clf):
    #Her algoritma aynı veri setini görsün diye
    np.random.seed(seed)
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.4)
    clf.fit(Xtrain, ytrain)
    ypred = clf.predict(Xtest)
    print(type(clf).__name__ + ': ', mean_squared_error(ypred,ytest))
    return type(clf).__name__, mean_squared_error(ypred,ytest)


boston_data = datasets.load_boston()
X = boston_data['data']
y = boston_data['target']

names = ['']*5
errors = [0]*5

print('Düşük daha iyi:')
names[0], errors[0] = quickTest(X,y,LinearRegression(fit_intercept=True))
names[1], errors[1] = quickTest(X,y,RandomForestRegressor(n_estimators=100))
names[2], errors[2] = quickTest(X,y,MLPRegressor(hidden_layer_sizes=(30,), max_iter=10000))
names[3], errors[3] = quickTest(X,y,GaussianProcessRegressor(kernel=DotProduct()+WhiteKernel()))
names[4], errors[4] = quickTest(X,y,GradientBoostingRegressor())

plt.bar(list(range(5)),errors)
plt.xticks(list(range(5)),names,rotation='vertical')
plt.show()